In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import gymnasium as gym
from pystk2_gymnasium import AgentSpec
from stk_actor.replay_buffer import SACRolloutBuffer
from stk_actor.wrappers import PreprocessObservationWrapper
import torch

class PolicyWrapper(torch.nn.Module):
    def __init__(self, policy_stb, dropout):
        super().__init__()
        self.shared = policy_stb
        self.dropout = torch.nn.Dropout(dropout)
    def forward(self, x):
        x = self.shared.features_extractor(x)
        x = self.shared.mlp_extractor.policy_net(x)
        x = self.dropout(x)
        x = self.shared.action_net(x)
        return x

vec_env = make_vec_env(
    "supertuxkart/flattened_multidiscrete-v0", 
    n_envs=1, 
    wrapper_class=lambda x : (PreprocessObservationWrapper(x)), 
    env_kwargs={
        'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"), 'difficulty':0,#'track':'abyss', #'num_kart':2, 'difficulty':0
    }
)
vec_env.close()

policy_stb = PPO('MlpPolicy', vec_env, policy_kwargs = dict(
    net_arch=[1024,1024,1024],
    activation_fn=torch.nn.Tanh,
)).policy

obs_dim = vec_env.observation_space.shape[0]
action_dims = [space.n for space in vec_env.action_space]

device = 'mps'
policy = PolicyWrapper(
    policy_stb, 0.2
).to(device)

obs_dim, action_dims

..:: Antarctica Rendering Engine 2.0 ::..


/Users/walidthekraken/VISA FRANCE/M2/RLD/stk_project/stk_actor/wrappers.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mean = torch.load(mod_path/'buffer_mean', 

(154, [5, 2, 2, 2, 2, 2, 7])

In [2]:
import joblib, random
from torch.optim import AdamW, Adam

batch_size = 1024 * 16
num_epochs = 1000
device = 'mps'

policy = policy.to(device)

optimizer = AdamW(policy.parameters(), lr=2e-3)
criterion = torch.nn.CrossEntropyLoss(reduction='none')

# buffer1 = joblib.load('all_tracks_buffer_steps_2mil')
# buffer2 = joblib.load('all_tracks_buffer_steps_1laps')

# buffer1 = joblib.load('all_tracks_buffer_steps_xx')
# buffer2 = joblib.load('all_tracks_buffer_steps_xxxx')
# buffer3 = joblib.load('all_tracks_buffer_steps_hardx')

# BEST 5 5 5
buffer4 = joblib.load('all_tracks_buffer_steps_diff2')
buffer5 = joblib.load('all_tracks_buffer_steps_diff2x')
buffer6 = joblib.load('all_tracks_buffer_steps_diff2xx')
buffer7 = joblib.load('all_tracks_buffer_steps_diff2xxx')

# batches = [[torch.tensor(b).to(device) for b in batch] for batch in list(buffer1.get_batches(batch_size, start_step_id))]
# batches.extend([[torch.tensor(b).to(device) for b in batch] for batch in list(buffer2.get_batches(batch_size, start_step_id))])

In [3]:
import numpy as np
# Combine buffers

start_step_id = 30

buffers = [buffer4,buffer5,buffer6, buffer7]
size = sum([b.size for b in buffers])
observations = torch.cat([b.observations[:b.size] for b in buffers], dim=0)
steps = torch.cat([b.steps[:b.size] for b in buffers], dim=0)
actions = torch.cat([torch.stack([actions for actions in b.actions]) for b in buffers], dim=1)

all_indices = np.arange(0, size)
all_indices = all_indices[steps[all_indices].flatten()>start_step_id]
observations = observations[all_indices]
actions = actions[:,all_indices]

_,unique_indices = np.unique(observations.numpy(), axis=0, return_index=True)
unique_indices = torch.tensor(unique_indices)

unique_observations = observations[unique_indices]
unique_actions = actions[:, unique_indices]

# unique_observations = observations
# unique_actions = actions

size = unique_observations.size(0)



# Result: unique_observations, unique_steps, unique_actions
def get_batches(batch_size, ):
    all_indices = np.arange(0, size)
    num_batches = int(np.ceil(len(all_indices) / batch_size))
    np.random.shuffle(all_indices)
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(all_indices))
        indices = all_indices[start_idx:end_idx]
        yield (
            unique_observations[indices],
            unique_actions[:,indices],
        )


In [4]:
# valid_obs = torch.concatenate(
#     [
#         buffer1.observations[:buffer1.size],
#         buffer2.observations[:buffer2.size],
#         # buffer3.observations[:buffer3.size]
#     ], dim=0
# )
mean = unique_observations.mean(dim=0).to(device)
std = unique_observations.std(dim=0).to(device)
mean.shape, std.shape, unique_observations.shape

(torch.Size([154]), torch.Size([154]), torch.Size([2000028, 154]))

In [5]:
policy.dropout = torch.nn.Identity()

In [ ]:
for epoch in range(num_epochs):

    policy.train()
    total_loss = 0
    num_batches = 0

    # if epoch %10 == 0:
        # batches = [[torch.tensor(b).to(device) for b in batch] for batch in list(buffer1.get_batches(batch_size, start_step_id))]
        # batches.extend([[torch.tensor(b).to(device) for b in batch] for batch in list(buffer2.get_batches(batch_size, start_step_id))])
    batches = [[torch.tensor(b).to(device) for b in batch] for batch in list(get_batches(batch_size))]
    action_correct = [0] * len(action_dims)
    action_total = [0] * len(action_dims)
    
    random.shuffle(batches)
    
    for batch in batches:
        # obs, actions, rewards, next_obs, prev_obs, dones, _, _ = batch
        obs, actions = batch
        obs = obs 
        # mean = obs.mean(dim=1, keepdim=True)
        # std = obs.std(dim=1, keepdim=True)
        obs = (obs - mean.unsqueeze(0)) / (std.unsqueeze(0) + 1e-8)
        obs = obs + torch.randn_like(obs) * 0.01

        actions = actions.permute(1, 0)

        outputs = policy(obs)
        optimizer.zero_grad()
        split_logits = torch.split(outputs, action_dims, dim=-1)
        
        losses = []
        # loss_weights = ([0,0,0,0,0,0,1]) # only train steering
        loss_weights = ([1,1,1,1,1,1,1]) # train all actions
        loss_weights = [x/sum(loss_weights) for x in loss_weights]
        for i in range(actions.size(1)):
            if loss_weights[i] ==0:
                continue
            loss_i = torch.nn.CrossEntropyLoss()(split_logits[i], actions[:,i])
            losses.append(loss_weights[i] * loss_i)
            
            # Compute accuracy
            predicted = torch.argmax(split_logits[i], dim=1)
            correct = (predicted == actions[:,i]).sum().item()
            total = actions.size(0)
            
            action_correct[i] += correct
            action_total[i] += total

            
        loss = sum(losses)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    avg_loss = total_loss / num_batches
    accuracies = [correct/(total or 1) * 100 for correct, total in zip(action_correct, action_total)]
    
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training Loss: {avg_loss:.4f}")
    for i, acc in enumerate(accuracies):
        print(f"Action {i} Accuracy: {acc:.2f}%")
    print("-" * 40)

print("Training complete!")

/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_3622/1452102833.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batches = [[torch.tensor(b).to(device) for b in batch] for batch in list(get_batches(batch_size))]


Epoch 1/1000
Training Loss: 0.2046
Action 0 Accuracy: 95.89%
Action 1 Accuracy: 97.31%
Action 2 Accuracy: 96.74%
Action 3 Accuracy: 98.14%
Action 4 Accuracy: 98.19%
Action 5 Accuracy: 98.24%
Action 6 Accuracy: 64.14%
----------------------------------------
Epoch 2/1000
Training Loss: 0.1208
Action 0 Accuracy: 98.04%
Action 1 Accuracy: 99.12%
Action 2 Accuracy: 98.44%
Action 3 Accuracy: 99.92%
Action 4 Accuracy: 99.96%
Action 5 Accuracy: 100.00%
Action 6 Accuracy: 75.19%
----------------------------------------
Epoch 3/1000
Training Loss: 0.1059
Action 0 Accuracy: 98.18%
Action 1 Accuracy: 99.23%
Action 2 Accuracy: 98.52%
Action 3 Accuracy: 99.92%
Action 4 Accuracy: 99.96%
Action 5 Accuracy: 100.00%
Action 6 Accuracy: 78.11%
----------------------------------------
Epoch 4/1000
Training Loss: 0.0974
Action 0 Accuracy: 98.30%
Action 1 Accuracy: 99.31%
Action 2 Accuracy: 98.60%
Action 3 Accuracy: 99.92%
Action 4 Accuracy: 99.96%
Action 5 Accuracy: 100.00%
Action 6 Accuracy: 79.80%
------

In [7]:
torch.save(policy.state_dict(),'policy_normed_1024_1024_1024_Tanh_statedict_2')
torch.save(mean,'buffer_mean_2')
torch.save(std,'buffer_std_2')